# Imports

In [1]:
import os
import sys
import pandas as pd

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

from helpers import data_analyzer as data_analyzer
from algorithms import angle_analyzer as angle_analyzer
from helpers import data_visualizer as data_visualizer
from helpers import optimize_thresholds as optimizer
import constants as constants

# Variables

In [2]:
CSV_FILE = "/Users/nicolas/Downloads/test_angle_analyzer.csv"

# Initialisation

In [3]:
Optimizer: optimizer.ThresholdOptimizer = optimizer.ThresholdOptimizer(
    CSV_FILE,
    constants.R_VALUE_WEIGHT,
    constants.P_VALUE_WEIGHT,
    constants.STD_ERROR_WEIGHT,
    constants.OPTIMIZATION_LIMIT,
    constants.OPTIMIZATION_STEPS,
)
AngleAnalyzer: angle_analyzer.AngleAnalyzer = angle_analyzer.AngleAnalyzer(
    CSV_FILE,
    constants.ANGLE_PAST_THRESHOLD,
    constants.ANGLE_FUTURE_THRESHOLD,
    constants.ANGLE_THRESHOLD,
    constants.LINEAR_REGRESSION_THRESHOLD,
)
DataAnalyzer: data_analyzer.DataAnalyzer = Optimizer.construct_data_analyzer()
Visualizer: data_visualizer.DataVisualizer = data_visualizer.DataVisualizer()

# Data Import

In [4]:
data: pd.DataFrame = AngleAnalyzer.read_csv_file()

# Optimization

In [5]:
optimization = Optimizer.optimize_thresholds(data, DataAnalyzer, AngleAnalyzer)

Iteration f = 10, p = 10 - 1 from 36:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 10
----> AVERAGE_R_VALUE: 0.5175449878654688
----> AVERAGE_P_VALUE: 6.701246656526688e-06
----> AVERAGE_STD_ERR: 0.03359707851462494
----> SCORE: 0.3071652744938218
--> Appending results...
--> Preparing next iteration...

Iteration f = 10, p = 15 - 2 from 36:
--> Testing thresholds...


/Users/nicolas/Dropbox/3_Bildung/2 KZO/Stichwortverzeichnis/0 Maturarbeit/6 Wettbewerbe/1_SJf/3_Coaching/2_Projekt/software/flight-analyzer/src/executor/../helpers/optimize_thresholds.py:184: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat(


--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 15
----> AVERAGE_R_VALUE: 0.5417478691429849
----> AVERAGE_P_VALUE: 3.2838485170747637e-06
----> AVERAGE_STD_ERR: 0.029394758423173736
----> SCORE: 0.3221082604889185
--> Appending results...
--> Preparing next iteration...

Iteration f = 10, p = 20 - 3 from 36:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 20
----> AVERAGE_R_VALUE: 0.5503222929845991
----> AVERAGE_P_VALUE: 3.975167172403912e-06
----> AVERAGE_STD_ERR: 0.027257190071738907
----> SCORE: 0.32746646423343384
--> Appending results...
--> Preparing next iteration...

Iteration f = 10, p = 25 - 4 from 36:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 25
----> AVERAGE_R_VALUE: 0.5814360041283861
----> AVERAGE_P_VALUE: 4.3109304254908796e-06
----> AVERAGE_STD_ERR: 0.025580510892788257
----> SCORE: 0.34630225810862525
--> Appending results...
--> Pre

In [6]:
Optimizer.export_to_csv(optimization)

# Visualisation

In [7]:
# Visualizer.visualize_optimization_linear_regression(optimization)
# Visualizer.visualize_optimization_score(optimization)
# Visualizer.visualize_optimization_rvalues(optimization)
# Visualizer.visualize_optimization_pvalues(optimization)
# Visualizer.visualize_optimization_stderrs(optimization)

# Report

In [8]:
print("Threasholds mit dem besten Score:")
print(f"--> past_threshold_optimized: {Optimizer.past_threshold_optimized}")
print(f"--> future_threshold_optimized: {Optimizer.future_threshold_optimized}")
print()

print("Optimierungsergebnisse (beste Kombination):")
print(f"--> ID: {optimization['score'].idxmax()}")
print(f"--> Score: {optimization['score'].max()}")
print(f"--> past_threshold: {optimization['angle_past_threshold'][optimization['score'].idxmax()]}")
print(f"--> future_threshold: {optimization['angle_future_threshold'][optimization['score'].idxmax()]}")

Threasholds mit dem besten Score:
--> past_threshold_optimized: 35
--> future_threshold_optimized: 35

Optimierungsergebnisse (beste Kombination):
--> ID: 35
--> Score: 0.41028994888438913
--> past_threshold: 35
--> future_threshold: 35


# System Info

In [9]:
print(f"@ Version {constants.VERSION}")
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")

@ Version 0.1.0
@ Author Nicolas Huber
@ Author Email info@nicolas-huber.ch
@ Author URL https://nicolas-huber.ch
@ GitHub URL https://github.com/nicolashuberIT/flight-analyzer
